In [3]:
from ncn.evaluation import Evaluator
from ncn.data import get_datasets

KeyError: 'widgetsnbextension'

In [ ]:
path_to_weights = "/home/timo/Downloads/NCN_7_19_10.pt"
path_to_data = "/home/timo/DataSets/KD_arxiv_CS/arxiv_data.csv"
data = get_datasets(path_to_data)

In [ ]:
evaluator = Evaluator(path_to_weights, data, evaluate=True, show_attention=False)

In [2]:
at_5 = evaluator.recall(5)
print(at_5)

NameError: name 'evaluator' is not defined